In [3]:
from Youtube_Transcript import y_transcript
from llama_index.core import Document
from llama_index.core.node_parser import CodeSplitter
from llama_index.core import SimpleDirectoryReader
import os
os.environ["GIT_PYTHON_REFRESH"] = "quiet"
from git import Repo
from langchain_community.document_loaders import UnstructuredURLLoader
from llama_index.core.node_parser import SentenceSplitter
from RAG import RAG,RAG_query
from vector_storing import vector_storing
from vector_load import vector_load
from llama_index.core import StorageContext, load_index_from_storage
from llama_index.vector_stores.chroma import ChromaVectorStore
import chromadb
from llama_index.llms.openai import OpenAI
from llama_index.core import VectorStoreIndex,SummaryIndex,StorageContext,SimpleKeywordTableIndex
from llama_index.embeddings.openai import OpenAIEmbedding
from git import Repo
from langchain_community.document_loaders.generic import GenericLoader
from langchain_community.document_loaders.parsers import LanguageParser
from langchain_text_splitters import Language
from langchain_text_splitters import RecursiveCharacterTextSplitter
import chromadb
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import StorageContext

def checker(data):

    # response_ext = {'url':'https://www.youtube.com/watch?v=RLUKn6dBhxU','video_id':'RLUKn6dBhxU','video_title':'7 Must-have Apps for Small Businesses (that we support)'}
    # response_ext = {'url':'https://github.com/stanford-futuredata/FrugalGPT','repo_name':'FrugalGPT'}
    response_ext = {'url':'https://scikit-learn.org/stable/auto_examples/model_selection/plot_grid_search_refit_callable.html#sphx-glr-auto-examples-model-selection-plot-grid-search-refit-callable-py'}


    if 'youtube' in response_ext['url'].lower():
        # print('hi')
        transcript_text = y_transcript(response_ext['video_id'])
        if 'video_title' in response_ext:
            document = Document(text=transcript_text,
                                    metadata = {"video_title": response_ext['video_title'], "video_url": response_ext['url']})
        else:
            document = Document(text=transcript_text,
                                    metadata = {"video_url": response_ext['url']})
        documents = [document]
        # print(documents)

    elif 'github' in response_ext['url'].lower():
        print(' in github')
        repo_name = response_ext['repo_name']
    
        repo_path = ".\\{}\\".format(repo_name)
        repo = Repo.clone_from(response_ext['url'], to_path=repo_path)
        loader = GenericLoader.from_filesystem(
            repo_path,
            glob="**/*",
            suffixes=[".py"],
            exclude=["**/non-utf8-encoding.py","setup.py"],
            parser=LanguageParser(language=Language.PYTHON, parser_threshold=500)
        )
        documents = loader.load()
        python_splitter = RecursiveCharacterTextSplitter.from_language(
        language=Language.PYTHON, chunk_size=2000, chunk_overlap=200
        )
        texts = python_splitter.split_documents(documents)
        db = chromadb.PersistentClient(path="./chroma_db")

        # create collection
        chroma_collection = db.get_or_create_collection("quickstart")

        # assign chroma as the vector_store to the context
        vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
        storage_context = StorageContext.from_defaults(vector_store=vector_store)

        # create your index
        index = VectorStoreIndex.from_documents(
            [documents], storage_context=storage_context
        )

        index = VectorStoreIndex.from_vector_store(
            vector_store, storage_context=storage_context
        )
        
        code_query_engine = index.as_query_engine()



        response = code_query_engine.query('can you list all the models that the author used and give the github repo of the code file where he used all the models')
        print(str(response))
        print()

        # response = code_query_engine.query('Can I get the repo link of scoring code')
        # print(str(response))
        # # RAG(nodes)
        

    else:
        loader = UnstructuredURLLoader(urls=[response_ext['url']],  show_progress_bar=True)
        loader = UnstructuredURLLoader(urls=[response_ext['url']],  show_progress_bar=True)
        documents = loader.load()
        document = Document(text=documents[0].page_content)
        parser = SentenceSplitter(chunk_size=1024)
        nodes = parser.get_nodes_from_documents([document])
        index = vector_storing("./embeddings/top_k","text-embedding-3-small","vector_store",nodes,"VectorStoreIndex")
        engione=index.as_query_engine()
        response = engione.query("what is candidate_idx ? can you explain ?")
        print(str(response))

        # print(data)
        print(nodes)



    # parser = SentenceSplitter(chunk_size=1024)
    # nodes = parser.get_nodes_from_documents(documents)
    # # print(nodes)



    # RAG(nodes)
    # response = RAG_query('can you summarise the entire video')
    # print(response)

# do webpage text stuff

In [4]:
checker('tset')

100%|██████████| 1/1 [00:00<00:00,  1.13it/s]


candidate_idx is the index of a model that has its test score greater than or equal to a specified threshold value. It is the index of a candidate model that meets the criteria of having a test score within 1 standard deviation of the best mean test score.
[TextNode(id_='1fb22c23-3351-460b-a5da-e614a913cd66', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='b130d70c-cab8-4c80-8f83-2f4a29a50d03', node_type=<ObjectType.DOCUMENT: '4'>, metadata={}, hash='c9fbe2189963bbfdc56008a1ac92226658e586b794d23f3577a71c740ef72259'), <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(node_id='4e9433b0-bf73-4c3a-945a-cfde38bb0151', node_type=<ObjectType.TEXT: '1'>, metadata={}, hash='65eb2542cf1369e9a2a66f2c1517ea30bc8f9222d8a9cc74a40615a0267ea44f')}, text='Install\n\nUser Guide\n\nAPI\n\nExamples\n\nCommunity\n\nGetting Started\n\nTutorial\n\nWhat\'s new\n\nGlossary\n\nDevelopment\n\nFAQ\n\nS

In [2]:
from fastapi import FastAPI
from pydantic import BaseModel

app = FastAPI()

class Message(BaseModel):
    content: str

@app.post("/receive_message")
async def receive_message(message: Message):
    return {"message_received": message.content}


In [1]:
from Youtube_Transcript import y_transcript
from llama_index.core import Document
from llama_index.core.node_parser import CodeSplitter
from llama_index.core import SimpleDirectoryReader
import os
os.environ["GIT_PYTHON_REFRESH"] = "quiet"
from git import Repo
from langchain_community.document_loaders import UnstructuredURLLoader
from llama_index.core.node_parser import SentenceSplitter
from RAG import RAG,RAG_query
from vector_storing import vector_storing
from vector_load import vector_load
from llama_index.core import StorageContext, load_index_from_storage
from llama_index.vector_stores.chroma import ChromaVectorStore
import chromadb
from llama_index.llms.openai import OpenAI
from llama_index.core import VectorStoreIndex,SummaryIndex,StorageContext,SimpleKeywordTableIndex
from llama_index.embeddings.openai import OpenAIEmbedding


def checker(data):

    # response_ext = data
    response_ext = {'url':'https://github.com/stanford-futuredata/FrugalGPT','repo_name':'FrugalGPT'}
    # response_ext = {'url':'https://docs.sweep.dev/blogs/chunking-2m-files'}


    if 'youtube' in response_ext['url'].lower():
        # print('hi')
        transcript_text = y_transcript(response_ext['url'].split('=')[-1])
        if 'video_title' in response_ext:
            document = Document(text=transcript_text,
                                    metadata = {"video_title": response_ext['video_title'], "video_url": response_ext['url']})
        else:
            document = Document(text=transcript_text,
                                    metadata = {"video_url": response_ext['url']})
        documents = [document]
        parser = SentenceSplitter(chunk_size=1024)
        nodes = parser.get_nodes_from_documents(documents)
        RAG(nodes)
        # print(documents)

    elif 'github' in response_ext['url'].lower():
        print(' in github')
        repo_name = response_ext['repo_name']
        # try:
        repo_path = ".\\{}\\".format(repo_name)
        repo = Repo.clone_from(response_ext['url'], to_path=repo_path)
        splitter = CodeSplitter('python')
        documents = SimpleDirectoryReader("./{}".format(repo_name),recursive=True,required_exts=[".py", ".ipynb"]).load_data()
        for i in documents:
            i.metadata['repo_name']=repo_name
            i.metadata['repo_link']=response_ext['url']+'/blob/master/'+i.metadata['file_path'][i.metadata['file_path'].find(repo_name)+len(repo_name):]
        nodes = splitter.get_nodes_from_documents(documents)
        
        # code_index = vector_storing("./embeddings/code","text-embedding-3-small","vector_store",nodes,"SimpleKeywordTableIndex")
        # index = VectorStoreIndex(
        #     nodes=nodes
        # )
        # index.storage_context.persist(persist_dir="./new_embeddings/code")
        # storage_context = StorageContext.from_defaults(persist_dir="./new_embeddings/code")
        # index = load_index_from_storage(storage_context)
        # print(index.index_struct)
        code_index = vector_storing("./embeddings/code","text-embedding-3-small","vector_store",nodes,"VectorStoreIndex")
        code_index = vector_load("./embeddings/code","text-embedding-3-small","vector_store","VectorStoreIndex")
            # code_query_engine = code_index.as_query_engine(similarity_top_k=15)

            # code_index = vector_load("./embeddings/code","text-embedding-3-small","vector_store","SimpleKeywordTableIndex")
            # # print(code_index.index_struct)
        code_query_engine = code_index.as_query_engine(similarity_top_k=15)

        response = code_query_engine.query('can you list all the models that the author used and give the github repo of the code file where he used all the models')
        print(str(response))
        print()

            # response = code_query_engine.query('Can I get the repo link of scoring code')
            # print(str(response))
            # # RAG(nodes)
        # except:
            # splitter = CodeSplitter('python')
            # documents = SimpleDirectoryReader("./{}".format(repo_name),recursive=True,required_exts=[".py", ".ipynb"]).load_data()
            # for i in documents:
            #     i.metadata['repo_name']=repo_name
            #     i.metadata['repo_link']=response_ext['url']+'/blob/master/'+i.metadata['file_path'][i.metadata['file_path'].find(repo_name)+len(repo_name):]
            # nodes = splitter.get_nodes_from_documents(documents)
            
            # code_index = vector_storing("./embeddings/code","text-embedding-3-small","vector_store",nodes,"SimpleKeywordTableIndex")
            # code_query_engine = code_index.as_query_engine(similarity_top_k=15)

            # code_index = vector_load("./embeddings/code","text-embedding-3-small","vector_store","SimpleKeywordTableIndex")
            # # print(code_index.index_struct)
            # code_query_engine = code_index.as_query_engine(similarity_top_k=15)

            # response = code_query_engine.query('can you list all the models that the author used and give the github repo of the code file where he used all the models')
            # print(str(response))
            # print()

            # response = code_query_engine.query('Can I get the repo link of scoring code')
            # print(str(response))
           

    else:
        loader = UnstructuredURLLoader(urls=[response_ext['url']],  show_progress_bar=True)
        documents = loader.load()
        parser = SentenceSplitter(chunk_size=1024)
        nodes = parser.get_nodes_from_documents(documents)
        RAG(nodes)
        # print(data)
        print(documents[0].metadata)



    
    # print(nodes)



    

# do webpage text stuff

c:\Users\harsh\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
checker('jsdfbjsd')

 in github
The author used the following models:
1. CohereAIModelProvider
2. OpenAIModelProvider
3. ForeFrontAIModelProvider
4. AnthropicModelProvider
5. APIModelProvider
6. OpenAIChatModelProvider
7. AI21ModelProvider

The GitHub repo of the code file where these models were used is: https://github.com/stanford-futuredata/FrugalGPT/blob/master/\src\service\modelservice.py

